In [ ]:
pip install transformers==4.11.3

In [ ]:
pip install datasets==2.2.2

In [ ]:
pip install seqeval

In [4]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import json
import transformers
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import datasets
from datasets import Features, Value, Sequence, load_metric, load_dataset
import torch

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path='/content/drive/MyDrive/ApResearch/'

In [7]:
class_names=[i for i in range(13)]
features=Features({'index': Value('int64'),'tokens': datasets.Sequence(datasets.Value("string")),"label": datasets.Sequence(datasets.features.ClassLabel(names=class_names))})

dataset = load_dataset('json', data_files={'train':path+'traindataNER.json', 'test': path+'testdataNER.json'},features=features, field='data')
dataset = dataset.rename_column('label', 'labels')
dataset

Using custom data configuration default-eb07787e88282924


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-eb07787e88282924/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'tokens', 'labels'],
        num_rows: 893
    })
    test: Dataset({
        features: ['index', 'tokens', 'labels'],
        num_rows: 158
    })
})

In [8]:
class_names=[i for i in range(11)]
features=Features({'index': Value('int64'),'tokens': datasets.Sequence(datasets.Value("string")),"label": datasets.Sequence(datasets.features.ClassLabel(names=class_names))})

dataset_merged = load_dataset('json', data_files={'train':path+'traindataNER_merged.json', 'test': path+'testdataNER_merged.json'},features=features, field='data')
dataset_merged = dataset_merged.rename_column('label', 'labels')

Using custom data configuration default-b80d8fb7c1656aea


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b80d8fb7c1656aea/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
m=0
for i in range(len(dataset['train'])):
  if len(dataset['train'][i]['tokens'])+2>m:
    m=len(dataset['train'][i]['tokens'])+2


In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=m+8,padding="max_length")

    labels = []
    for i, label in enumerate(examples[f"labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
 def trainNER(model, dataset,num_labels, learning_rate,per_device_batch_size,num_train_epochs,weight_decay,label_list):
  dt = dataset.map(tokenize_and_align_labels, batched=True)
  data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
  model = AutoModelForTokenClassification.from_pretrained(model, num_labels=num_labels)
  training_args = TrainingArguments(
      output_dir="./results",
      evaluation_strategy="epoch",
      learning_rate=learning_rate,
      per_device_train_batch_size=per_device_batch_size,
      per_device_eval_batch_size=per_device_batch_size,
      num_train_epochs=num_train_epochs,
      do_eval=False,
      do_predict=False
  )

  def compute_metrics(p):
      predictions, labels = p
      predictions = np.argmax(predictions, axis=2)

      # Remove ignored index (special tokens)
      true_predictions = [
          [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
          for prediction, label in zip(predictions, labels)
      ]
      true_labels = [
          [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
          for prediction, label in zip(predictions, labels)
      ]

      results = metric.compute(predictions=true_predictions, references=true_labels)
      return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
      }

  metric = load_metric("seqeval")

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=dt["train"],
      eval_dataset=dt["test"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics
  )
  trainer.train()

  predictions, labels, _ = trainer.predict(dt["test"])
  predictions = np.argmax(predictions, axis=2)

  # Remove ignored index (special tokens)
  true_predictions = [
      [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]

  results = metric.compute(predictions=true_predictions, references=true_labels)
  return results, model



In [12]:
lr=[0.00005]
per_device_batch_size=[1]
epochs=[2,4,6,8]
weight_decay=[0.025,0.005]


In [13]:
hyper=np.array(np.meshgrid(lr, per_device_batch_size, epochs, weight_decay)).T.reshape(-1, 4)
hyper.shape

(8, 4)

In [14]:
r_=0
ii=0
for i in range(len(hyper)):
  tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-pretrain")
  r,model=trainNER("yiyanghkust/finbert-pretrain", dataset,13,hyper[i,0],int(hyper[i,1]),int(hyper[i,2]),hyper[i,3],label_list=['O', 'B-FRM','I-FRM','B-PRD_S','I-PRD_S','B-AMNT','I-AMNT','B-PSN','I-PSN','B-MRKT','I-MRKT','B-LOC','I-LOC'])
  if r['overall_f1']>r_:
    r_=r['overall_f1']
    ii=i
    model_=model
torch.save(model_.state_dict(), path+"finbert_full_"+str(hyper[ii,0])+"_"+str(hyper[ii,1])+"_"+str(hyper[ii,2])+str(hyper[ii,3]))

Downloading:   0%|          | 0.00/359 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.374400,0.209332,0.577114,0.489451,0.529680,0.939781
2,0.131400,0.172324,0.666667,0.675105,0.670860,0.955683
3,0.062000,0.207088,0.634855,0.645570,0.640167,0.952815
4,0.024800,0.240546,0.661088,0.666667,0.663866,0.953337
5,0.017800,0.258356,0.693617,0.687764,0.690678,0.956465
6,0.006400,0.261531,0.678862,0.704641,0.691511,0.955162


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: index, tokens.
***** Running Evaluation *****
  Num examples = 158
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-15

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/yiyanghkust/finbert-pretrain/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/52a95139f4c3367976e7045d2f955bf82242a485e8506539b49838ca3c825654.ddd3650c75e2551777ba1291415522ee7288b429a28ea24681ddf54025802027
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30873
}

loading file https://huggingface.co

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.372200,0.192876,0.624365,0.518987,0.566820,0.944473
2,0.131700,0.163348,0.591093,0.616034,0.603306,0.948644
3,0.059800,0.247003,0.706186,0.578059,0.635731,0.951251
4,0.023300,0.255146,0.617188,0.666667,0.640974,0.949948
5,0.022700,0.273214,0.678715,0.713080,0.695473,0.956986
6,0.012200,0.286707,0.655172,0.721519,0.686747,0.956726
7,0.004200,0.299968,0.676000,0.713080,0.694045,0.959593
8,0.003600,0.307903,0.656250,0.708861,0.681542,0.957508


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: index, tokens.
***** Running Evaluation *****
  Num examples = 158
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-15

In [ ]:
r_=0
ii=0
for i in range(len(hyper)):
  tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-pretrain")
  r,model=trainNER("yiyanghkust/finbert-pretrain", dataset_merged,11,hyper[i,0],int(hyper[i,1]),int(hyper[i,2]),hyper[i,3],label_list=['O', 'B-ORG','I-ORG','B-PSM','I-PSM','B-AMNT','I-AMNT','B-PSN','I-PSN','B-LOC','I-LOC'])
  if r['overall_f1']>r_:
    r_=r['overall_f1']
    ii=i
    model_=model
torch.save(model_.state_dict(), path+"finbert_merged_"+str(hyper[ii,0])+"_"+str(hyper[ii,1])+"_"+str(hyper[ii,2])+str(hyper[ii,3]))

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/yiyanghkust/finbert-pretrain/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/52a95139f4c3367976e7045d2f955bf82242a485e8506539b49838ca3c825654.ddd3650c75e2551777ba1291415522ee7288b429a28ea24681ddf54025802027
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30873
}

loading file https://huggingface.co

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/yiyanghkust/finbert-pretrain/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/52a95139f4c3367976e7045d2f955bf82242a485e8506539b49838ca3c825654.ddd3650c75e2551777ba1291415522ee7288b429a28ea24681ddf54025802027
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.304300,0.193571,0.649457,0.678977,0.663889,0.933725
2,0.087100,0.207193,0.682320,0.701705,0.691877,0.940964


...

Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: index, tokens.
***** Running Evaluation *****
  Num examples = 158
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-15

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/yiyanghkust/finbert-pretrain/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/52a95139f4c3367976e7045d2f955bf82242a485e8506539b49838ca3c825654.ddd3650c75e2551777ba1291415522ee7288b429a28ea24681ddf54025802027
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30873
}

loading file https://huggingface.co

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.315300,0.209449,0.588556,0.613636,0.600834,0.926261
2,0.108000,0.224332,0.664042,0.718750,0.690314,0.938249


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: index, tokens.
***** Running Evaluation *****
  Num examples = 158
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-15

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.315300,0.209449,0.588556,0.613636,0.600834,0.926261
2,0.108000,0.224332,0.664042,0.718750,0.690314,0.938249
